In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Input, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Load the data
df = pd.read_csv('food_supplements.csv')

In [3]:
# Convert 'Sold_date' to datetime format and set as index
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')
df.set_index('Sold_date', inplace=True)

In [4]:
# Aggregate sales data on a weekly basis for each product
weekly_data = df.pivot_table(values='Sold_quantity', index='Sold_date', 
                             columns='Product_details', aggfunc='sum').resample('W').sum()

In [5]:
# Normalize the sales data
scaler = MinMaxScaler()
weekly_data_scaled = scaler.fit_transform(weekly_data)
weekly_data_scaled = pd.DataFrame(weekly_data_scaled, index=weekly_data.index, columns=weekly_data.columns)


In [6]:
# Prepare data for LSTM
def create_sequences(data, sequence_length):
    x = []
    y = []
    for i in range(len(data) - sequence_length):
        x.append(data.iloc[i:i + sequence_length].values)
        y.append(data.iloc[i + sequence_length].values)
    return np.array(x), np.array(y)



In [7]:
sequence_length = 2  # Number of weeks used to predict the next week
x_lstm, y_lstm = create_sequences(weekly_data_scaled, sequence_length)

# Display the input sequences
# Display the input sequences
print("Input Sequences Shape: (162, 2, 92)")
print("Output Sequences Shape: (162, 92)")
print("Input Sequences:")

# Loop through each sequence and print it
for i in range(len(x_lstm)):
    print(f"Sequence {i + 1}")
    print(x_lstm[i])
    print()

# Display the corresponding output data for each sequence
print("Output Sequences:")
for i in range(len(y_lstm)):
    print(f"Sequence {i + 1} Output")
    print(y_lstm[i])
    print()





Input Sequences Shape: (162, 2, 92)
Output Sequences Shape: (162, 92)
Input Sequences:
Sequence 1
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.08403361
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.      

[0.         0.23255814 0.         0.         0.17647059 0.
 0.15384615 0.58415842 0.         0.         0.         0.68965517
 0.34615385 0.125      0.         0.36363636 0.         0.55
 0.6        0.08       0.         0.11666667 0.34188034 0.25
 0.         0.         0.3961039  0.14285714 0.         0.
 0.         0.02083333 0.09090909 0.         0.         0.
 0.         0.65217391 0.         0.         0.2        0.
 0.11206897 0.52631579 0.         0.09615385 0.         0.22826087
 0.45967742 0.         0.         0.1875     0.         0.
 0.         0.34883721 0.45177665 0.         0.         0.04201681
 0.71428571 0.35622318 0.         0.         0.         0.5
 0.         0.05633803 0.         0.         0.5        0.28571429
 0.35       0.         0.1        0.         0.         0.26923077
 0.         0.         0.26086957 0.         0.         0.
 0.         0.         0.         0.         0.03703704 0.15315315
 0.         0.        ]

Sequence 156 Output
[0.         0.232

In [8]:
# Define the LSTM model to extract features
lstm_model = Sequential([
    Input(shape=(sequence_length, len(weekly_data_scaled.columns))),
    LSTM(92, activation='tanh', recurrent_activation='sigmoid', return_sequences=False),
])

In [9]:
# Compile the LSTM model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

In [10]:
# Fit the LSTM model
history = lstm_model.fit(x_lstm, y_lstm, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - loss: 0.0440 - val_loss: 0.0618
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0423 - val_loss: 0.0571
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0389 - val_loss: 0.0533
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0360 - val_loss: 0.0501
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0347 - val_loss: 0.0475
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0318 - val_loss: 0.0454
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0291 - val_loss: 0.0439
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0279 - val_loss: 0.0429
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0276 - val_loss: 0.0424
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0253 - val_loss: 0.0423
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0253 - val_loss: 0.0422
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0239 - val_lo

Epoch 70/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0157 - val_loss: 0.0407
Epoch 71/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0161 - val_loss: 0.0408
Epoch 72/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0161 - val_loss: 0.0405
Epoch 73/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0156 - val_loss: 0.0405
Epoch 74/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0157 - val_loss: 0.0406
Epoch 75/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0158 - val_loss: 0.0407
Epoch 76/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0156 - val_loss: 0.0409
Epoch 77/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0158 - val_loss: 0.0412
Epoch 78/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0158 - val_loss: 0.0413
Epoch 79/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0153 - val_loss: 0.0415
Epoch 80/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0151 - val_loss: 0.0415
Epoch 81/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0147 - 

In [11]:
# Extract features using LSTM
features = lstm_model.predict(x_lstm)



# Print the features of the first three samples to see what they look like
print("Features of the first three samples:\n", features[:3])


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Features of the first three samples:
 [[ 1.58590712e-02  4.46307436e-02  9.24054906e-03 -3.85440327e-03
   5.08583933e-02 -1.41522186e-02  3.40352468e-02 -1.26288822e-02
   2.37617251e-02 -5.44052664e-03 -2.12144032e-02 -1.57386828e-02
   4.30267453e-02 -3.44051048e-02 -4.21711057e-03  1.06492750e-02
  -2.76555493e-03  1.04300700e-01  6.63873414e-03  1.12216510e-01
   3.90478000e-02  9.03796405e-02 -1.26963807e-02  2.53184009e-02
   1.92346086e-03 -3.01912837e-02  4.72671241e-02 -1.36811445e-02
   1.02630882e-02  1.11028384e-02  3.23120803e-02  2.42671762e-02
   8.48456398e-02  7.24903047e-02  8.45495909e-02  6.44529844e-03
   1.67588256e-02  3.95873971e-02 -2.01798324e-02  5.08730225e-02
   4.20421772e-02  6.66966140e-02  9.71830636e-03  1.14649080e-01
   4.40579168e-02  4.05820645e-02  1.43232348e-04  1.56813562e-01
   1.23471627e-02 -2.41143964e-02  1.80250127e-02  3.99709269e-02
   1.50654828e-02 -5.55430306e-03 -4.52152733e-03  2.55334768e-02


In [12]:
# Enhanced CNN model with additional complexity and dropout for regularization
input_shape = (features.shape[1], features.shape[2]) if len(features.shape) == 3 else (features.shape[1], 1)  
cnn_model = Sequential([
    Input(shape=input_shape),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(len(weekly_data.columns))
])

cnn_model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])


In [13]:
# Model summary
cnn_model_enhanced.summary()

NameError: name 'cnn_model_enhanced' is not defined

In [ ]:
# Fit the CNN model
history = cnn_model.fit(features, y_lstm, epochs=100, batch_size=32, validation_split=0.2)

# Predictions made by the CNN model
scaled_predictions = cnn_model_enhanced.predict(features)


In [ ]:
# Display RMSE
print("Root Mean Squared Error (RMSE):")
print(history.history['root_mean_squared_error'])

In [ ]:
# Assuming `y_lstm` and `scaled_predictions` have the same second dimension (number of features or products)
if scaled_predictions.shape[1] == y_lstm.shape[1]:
    actual_predictions = scaler.inverse_transform(scaled_predictions)
    actual_targets = scaler.inverse_transform(y_lstm)
else:
    print("Error: Mismatch in dimensions between predictions and actuals.")


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Calculate RMSE in original units
rmse_original = sqrt(mean_squared_error(actual_targets, actual_predictions))
print("RMSE on original scale:", rmse_original)
